In [1]:
import json
import re
from pathlib import Path

In [2]:
M3U_FILE = "tung_iptv.m3u"
OUTPUT_JSON = "tung_iptv.json"

def slugify(text):
    return re.sub(r'[^a-z0-9]+', '-', text.lower()).strip('-')

def stream_type(url):
    return "hls" if url.endswith(".m3u8") else "direct"

def parse_m3u(path):
    groups = {}
    current_info = None

    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        for line in f:
            line = line.strip()
            if not line or line.startswith("#EXTM3U"):
                continue

            if line.startswith("#EXTINF"):
                group = re.search(r'group-title="([^"]+)"', line)
                logo = re.search(r'tvg-logo="([^"]+)"', line)
                name = line.split(",", 1)[-1].strip()

                current_info = {
                    "group": group.group(1) if group else "Others",
                    "name": name,
                    "logo": logo.group(1) if logo else None
                }

            elif current_info and not line.startswith("#"):
                group_name = current_info["group"]
                groups.setdefault(group_name, [])

                channel_id = slugify(current_info["name"])

                groups[group_name].append({
                    "id": channel_id,
                    "name": current_info["name"],
                    "label": None,
                    "image": {
                        "url": current_info["logo"],
                        "type": "contain",
                        "width": 64,
                        "height": 64
                    } if current_info["logo"] else None,
                    "display": "default",
                    "type": "single",
                    "enable_detail": False,
                    "sources": [{
                        "id": "source-1",
                        "name": "Main",
                        "image": None,
                        "contents": [{
                            "id": "content-1",
                            "name": "Live",
                            "image": None,
                            "grid_number": 1,
                            "streams": [{
                                "id": f"{channel_id}-stream",
                                "name": "Live",
                                "image": None,
                                "stream_links": [{
                                    "id": f"{channel_id}-link-1",
                                    "name": "Server 1",
                                    "url": line,
                                    "type": stream_type(line),
                                    "default": True,
                                    "enableP2P": False,
                                    "subtitles": None,
                                    "remote_data": None,
                                    "request_headers": None,
                                    "comments": None
                                }]
                            }]
                        }],
                        "remote_data": None
                    }]
                })

                current_info = None

    return groups

def build_json(groups):
    return {
        "id": "tung-iptv",
        "name": "Tung IPTV",
        "description": "Sports",
        "url": "",
        "color": "#1E90FF",
        "image": None,
        "grid_number": 2,
        "groups": [
            {
                "id": slugify(group),
                "name": group,
                "display": "vertical",
                "image": None,
                "grid_number": 3,
                "enable_detail": False,
                "channels": channels
            }
            for group, channels in groups.items()
        ]
    }

if __name__ == "__main__":
    groups = parse_m3u(M3U_FILE)
    data = build_json(groups)

    with open(OUTPUT_JSON, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2)

    print(f"✅ Converted {len(groups)} groups → {OUTPUT_JSON}")

✅ Converted 4 groups → tung_iptv.json
